## Introduction 

Dis lesson go cover: 
- Wetin be function calling and how e dey useful 
- How to create function call using OpenAI 
- How to put function call inside application 

## Learning Goals 

After you finish dis lesson, you go sabi how to and understand: 

-  Why you go use function calling 
- How to setup Function Call using OpenAI Service 
- How to design beta function calls for how your application go take use am


## Understanding Function Calls 

For dis lesson, we wan build one feature for our education startup wey go allow users use chatbot to find technical courses. We go recommend courses wey fit their skill level, current role and technology wey dem dey interested for. 

To complete dis, we go use combination of: 
 - `OpenAI` to create chat experience for the user
 - `Microsoft Learn Catalog API` to help users find courses based on wetin the user request 
 - `Function Calling` to carry the user's query come send am go function to make the API request. 

To start, make we look why we go want use function calling for the first place: 

print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get new response from GPT wey fit see the function response


print(second_response.choices[0].message)


### Why Function Calling 

If you don complete any oda lesson for dis course, you fit don sabi di power wey dey for using Large Language Models (LLMs). Hope say you fit also see some of dia limitations too. 

Function Calling na one feature of di OpenAI Service wey dem design to solve dis kain wahala:

Inconsistent Response Formatting:
- Before function calling, responses from one large language model no get structure and dem no consistent. Developers for write complex validation code to handle every variation wey dey for di output.

Limited Integration with External Data:
- Before dis feature, e hard to put data from oda parts of one application inside chat context.

By standardizing response formats and making e easy to join external data, function calling dey make development simple and e reduce di need for extra validation logic.

Users no fit get answers like "Wetin be di current weather for Stockholm?". Dis na because models dey limited to di time wey dem train di data.

Make we look di example wey dey below wey show dis problem:

Make we talk say we wan create one database of student data so dat we fit suggest di correct course to dem. Below we get two descriptions of students wey similar well well for di data wey dem get.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

We wan send dis to an LLM make e parse di data. Dis one fit later dey use for our application to send am go API or store am for database.

Make we create two identical prompts wey we go tell di LLM wetin kind information we dey interested for:


We wan send dis to one LLM make e fit parse di parts wey important to our product. So we fit create two identical prompts to instruct di LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Afta we don create dis two prompts, we go send dem go LLM by using `openai.ChatCompletion`. We dey store di prompt for di `messages` variable and assign di role to `user`. Dis na to mimic message wey user dey write go chatbot.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

Now we fit send both requests go the LLM and check the response we receive.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Even tho di prompts na di same and di descriptions dey similar, we fit get different formats for di `Grades` property.

If you run di cell wey dey above many times, di format fit be `3.7` or `3.7 GPA`.

Dis na because di LLM dey take unstructured data for di form of di written prompt and e go return unstructured data too. We need make we get structured format so dat we go sabi wetin to expect when we dey store or use dis data.

By using functional calling, we fit make sure say we go receive structured data back. When we dey use function calling, di LLM no really dey call or run any functions. Instead, we dey create structure for di LLM to follow for im responses. We go then use those structured responses to sabi which function to run for our applications.


![Function Calling Flow Diagram](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.pcm.png)


We fit den take wetin di function return come send am back to di LLM. Di LLM go den respond wit natural language to answer di user query.


### Use Cases for using function calls 

**Calling External Tools**  
Chatbots dey very good for giving answers to questions wey users ask. By using function calling, the chatbots fit use messages from users to complete certain tasks. For example, one student fit ask the chatbot to "Send email to my instructor saying I need more assistance with this subject". Dis one fit make function call to `send_email(to: string, body: string)`


**Create API or Database Queries**  
Users fit find information by using natural language wey go convert into formatted query or API request. Example be say one teacher fit ask "Who are the students that completed the last assignment" wey fit call function wey dem name `get_completed(student_name: string, assignment: int, current_status: string)`


**Creating Structured Data**  
Users fit take one block of text or CSV and use the LLM to extract important information from am. For example, one student fit convert Wikipedia article about peace agreements to create AI flash cards. Dis one fit happen by using function wey dem call `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Creating Your First Function Call 

Di process wey you go take create function call get 3 main steps: 
1. Call di Chat Completions API wit list of your functions and user message 
2. Read di model response to perform action like execute function or API Call 
3. Make another call to Chat Completions API wit di response from your function to use dat info create response to di user. 


![Flow of a Function Call](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.pcm.png)


### Elements of a function call 

#### Users Input 

Di first step na to create user message. Dis fit dey dynamically assigned by taking di value of text input or you fit assign value here. If na your first time to dey work with Chat Completions API, we need to define di `role` and di `content` of di message. 

Di `role` fit be either `system` (wey dey create rules), `assistant` (di model) or `user` (di end-user). For function calling, we go assign am as `user` plus example question. 


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Creating functions. 

Next we go define one function and the parameters of that function. We go use just one function here wey dem call `search_courses` but you fit create plenty functions.

**Important** : Functions dey inside the system message to the LLM and dem go count for the amount of tokens wey you get.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definitions** 

Di function definition structure get plenti levels, each get im own properties. Dis na di breakdown of di nested structure:

**Top Level Function Properties:**

`name` -  Di name of di function wey we want make e dey called. 

`description` - Dis na di description of how di function dey work. Here e important to dey specific and clear 

`parameters` - List of values and format wey you want make di model produce for im response 

**Parameters Object Properties:**

`type` -  Di data type of di parameters object (usually "object")

`properties` - List of di specific values wey di model go use for im response 

**Individual Parameter Properties:**

`name` - Implicitly defined by di property key (e.g., "role", "product", "level")

`type` - Di data type of dis specific parameter (e.g., "string", "number", "boolean") 

`description` - Description of di specific parameter 

**Optional Properties:**

`required` - Array wey dey list which parameters dey required for di function call to complete 


### Making the function call 
After we don define function, now we need put am for the call to the Chat Completion API. We dey do dis by adding `functions` to the request. For dis case, na `functions=functions`. 

E still get option to set `function_call` to `auto`. Dis one mean say we go allow the LLM decide which function suppose call based on the user message instead make we assign am ourselves.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Now make we look di response and see how e take format: 

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

You fit see say di name of di function don call and from di user message, di LLM fit find di data wey go match di arguments of di function. 


## 3.Integrating Function Calls into an Application. 


After we don test the formatted response from the LLM, now we fit integrate dis into an application. 

### Managing the flow 

To integrate dis into our application, make we follow dis steps: 

First, make we call the OpenAI services and store the message for one variable wey dem call `response_message`. 


In [ ]:
response_message = response.choices[0].message

Now we go define di function wey go call di Microsoft Learn API to get list of courses:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



As best practice, we go den check if di model wan call function. After dat, we go create one of di available functions and match am to di function wey dem dey call.  
We go den take di arguments of di function and map dem to arguments from di LLM.

Lastly, we go add di function call message and di values wey di `search_courses` message return. Dis one go give di LLM all di information wey e need to  
respond to di user using natural language.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



Now we go send di updated message go di LLM so we fit receive natural language response instead of API JSON formatted response.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Code Challenge 

Great work! To continue your learning of OpenAI Function Calling you fit build: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - More parameters of the function wey fit help learners find more courses. You fit find the available API parameters here: 
 - Create another function call wey go take more information from the learner like their native language 
 - Create error handling wen the function call and/or API call no return any suitable courses 


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:
Dis document na AI translation service [Co-op Translator](https://github.com/Azure/co-op-translator) wey translate am. Even though we dey try make am correct, abeg sabi say automated translation fit get some mistakes or no too correct. The original document wey e dey for im own language na im be the correct one. If na serious matter, e better make person wey sabi translate am well do am. We no go responsible for any wahala or wrong understanding wey fit happen because of this translation.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
